## COG403: Problem 2 of Problem Set 1: Betas and Homophones

### All 3 problems for Problem Set 1 Due 4 October 2018

Imagine you are a child learning English. You are building up your vocabulary, but you are struggling with homophones (words that mean different things but sound the same). In particular, you are working on learning the difference between *for* and *four*, both of which are phonetically [fɔɹ]. (This is IPA. For more information, see https://en.wikipedia.org/wiki/International_Phonetic_Alphabet).

You assume that there is a parameter, $\theta$, that controls how often [fɔɹ] conveys each meaning (*for* and *four*). This assumption can be formalized as the graphical model shown below. Each $X_i$ represents a sentence that uses a word that sounds like [fɔɹ]. $X_i$ takes on value 1 if [fɔɹ] in sentence $i$ means *for* and 0 if it means *four*. The assumption underlying this graphical model is that there is some unobserved value of $\theta$ and that for each sentence, a biased coin is flipped to determine whether a word that sounds like [fɔɹ] will mean *for* (with probability $\theta$) or mean *four* (with probability $1- \theta$ ). You would like to learn the value of $\theta$ by observing sentences that contain words that are pronounced [fɔɹ].

We will model learning [fɔɹ] based on corpus data.

![betas graphical model](https://notebooks.azure.com/juliawatson/libraries/q2-betas-and-homophones/raw/grapical_model_betas.png)

### (a) 

Write a function to load in the Brent Ratner child-directed speech corpus and return a dictionary mapping each word type to its frequency in the corpus. This corpus is stored in `data/brent-ratner-corpus.txt` in the library for this notebook.

In [2]:
from collections import Counter

def compute_word_counts(path_to_file):
    """
    path_to_file: string -- the path to the corpus file
    
    Return a dictionary mapping each unique type in the corpus at path_to_file
    to the number of times it occurs in the corpus. Make sure to convert words
    to all lowercase to get unique types.
    """
    
    # Student TODO: implement this function
    file_object = open(path_to_file, 'r')
    
    word_dict = dict()
    
    for sentence in file_object:
        for word in sentence.split():
            word = word.lower()
            
            if word not in word_dict:
                word_dict[word] = 1
            
            elif word in word_dict:
                word_dict[word] += 1
    
    return word_dict

corpus_counts = compute_word_counts('data/brent-ratner-corpus.txt')

### (b)

The estimate of $\theta$ that maximizes your posterior distribution is called the MAP estimate (MAP stands for maximum a posteriori), which we refer to as $\hat{\theta}$. If your posterior distribution is $Beta(a_1 + k_1, a_2 + k_2)$, then you can compute $\hat{\theta}$ using the formula $\frac{a_1+k_1−1}{(a_1+k_1−1)+(a_2+k_2−1)}$ . Use this formula to compute $\hat{\theta}$ after observing child-directed utterances below (taken from the Brent Ratner Corpus$^1$ from CHILDES$^2$):
1. This is food **for** the dragon.
2. One block, two blocks, three blocks, **four** blocks.
3. Thank you **for** the phone.
4. What do you want to get **for** your birthday?

Assume an initial prior distrubution of $Beta(1, 1)$. You may do this by hand or write Python code. If you choose to do it by hand, be sure to show your work.

#### (b) Solution

Let $k_1$ be the number of times "for" is meant, and $k_2$ the number of times "four" is meant. In that case, $k_1 = 3, k_2 = 1$. Thus, 

\begin{align*}
	\hat{\theta} &= \frac{a_1 + k_1 - 1}{(a_1 + k_1 - 1) + (a_2 + k_2 -1)}\\
				 &= \frac{1 + 3 - 1}{(1 + 3 - 1) + (1 + 1 - 1)}\\
				 &= \frac{3}{4}
\end{align*}


### (c)

You have two friends, Jack and Jill, who are also trying to learn the meaning of [fɔɹ]. Their learning biases are different from yours. Jack has a prior distribution of $Beta(10,10)$ and Jill has a prior distribution of $Beta(100,100)$. After observing the utterances from part b, what are the parameters of their posterior distributions? What are their MAP estimates of $\theta$? You may do this by hand or write Python code. If you choose to do it by hand, be sure to show your work.

#### (c) Solution

Let $k_1$ be the number of times "for" is meant, and $k_2$ the number of times "four" is meant. In that case, $k_1 = 3, k_2 = 1$. Jack has a posterior distribution of $Beta(13, 11)$ and a MAP estimate of $\theta$ of: 

\begin{align*}
	\hat{\theta} &= \frac{a_1 + k_1 - 1}{(a_1 + k_1 - 1) + (a_2 +k_2 -1)}\\
				 &= \frac{10 + 3 - 1}{(10 + 3 - 1) + (10 + 1 - 1)}\\
				 &= \frac{6}{11} \approx 0.54545
\end{align*}
	

For Jill the posterior distribution is $Beta(103, 101)$ with a a MAP estimate of $\theta$ is: 

\begin{align*}
	\hat{\theta} &= \frac{a_1 + k_1 - 1}{(a_1 + k_1 - 1) + (a_2 +k_2 -1)}\\
				 &= \frac{100 + 3 - 1}{(100 + 3 - 1) + (100 + 1 - 1)}\\
				 &= \frac{102}{202} \approx 0.50495
\end{align*}



### (d)

Write a function that uses the word frequencies computed in part a above to compute the probability of word given a list of its homophones. This probability will serve as the "true" $\theta$ value -- the value that the child is seeking to learn from the sample of data they're exposed to.

In [4]:
def compute_theta(corpus_counts, word1, homophone_list):
    """
    corpus_counts: dict of str->int, mapping words to their frequencies
    word1: str
    homophone_list: list of words that sound the same as word1. word1 must be in homophone_list.
    
    Return the probability of word1 given that a word from homophone_list occurred.
    """
    # student TODO: implement this function
    
    count_word1 = corpus_counts[word1]
    
    N = 0
    for homophone in homophone_list:
        if homophone in corpus_counts:
            N += corpus_counts[homophone]

    return count_word1 / N

true_theta = compute_theta(corpus_counts, "for", ["for", "four"])

0.9074074074074074


In this case, true theta comes to 0.9074.

### (e)

Compute the squared error of each of the MAP estimates (for you, Jack, and Jill) based on the true $\theta$ computed in part d. Who had the lowest squared error: you, Jack, or Jill? You may do this by hand or write Python code. If you choose to do it by hand, be sure to show your work.

#### (e) Solution

The formula for squared error is:
$$E = (\text{real-value - estimated-value})^2 $$

Thus:

My squared error is $(\frac{196}{216} - \frac{3}{4})^2 = 0.0248 $

Jack's squared error is $(\frac{196}{216} - \frac{6}{11})^2 = 0.131 $

Jill's squared error is $(\frac{196}{216} - \frac{102}{202})^2 = 0.162$

Thus I had the lowest squared error.


### (f)

Write a function `generate_corpus` that creates a random corpus that you and your friends might encounter based on the theta value computed in part d. For any integer $n$, your command should return a 1-by-n vector consisting of ones (uses of [fɔɹ] that mean *for*) and zeros (uses of [fɔɹ] that mean *four*), where 1 occurs approximately $\theta$ fraction of the time and 0 occurs approximately (1 - $\theta$) fraction of the time. (Hint: using `numpy.random.rand(n)` will give you a vector of $n$ random numbers uniformly sampled from $[0,1)$. How can you use this to generate a list of ones and zeros where 1 occurs $\theta$ fraction of the time?)

In [5]:
import numpy as np

def generate_corpus(real_theta, n):
    """
    n: int, the length of the generated random vector
    
    Return a vector filled with ones (uses of *for*) and
    zeros (uses of *four*), where 1 occurs approximately
    real_theta fraction of the time and 0 occurs
    approximately (1 - real_theta) fraction of the time.
    """
        
    random_vec = np.random.rand(n)
    
    for index in range(len(random_vec)):
        
        if random_vec[index] <= real_theta:
            random_vec[index] = 1
        else:
            random_vec[index] = 0

    return list(random_vec)

### (g)

Write a function `learn` to simulate a learner. Your function should take in a number $n$, the parameters $a_1$ and $a_2$ of the Beta prior distribution, and the true $\theta$ value. It should first generate a random corpus of length $n$ (using `generate_corpus` from f), then use this corpus together with the prior to find the parameters of the posterior distribution, and finally use those parameters to compute the MAP estimate $\hat{\theta}$ and the squared error of this estimate. Your function should return the MAP estimate as well as the squared error.

Test out this function by calling it using:
 * `a_1 = a_2 = 1`
 * `true_theta` = the true theta value for *for* (computed in part d)
 * `n = 100`

In [6]:
def learn(a1, a2, n, true_theta):
    """
    a1: int -- parameter for prior Beta distribution
    a2: int -- parameter for prior Beta distribution
    n: int -- number of samples to use to update Beta distribution
    true_theta: float -- the theta value we want to model. We use it to generate a corpus.

    Return MAP theta value and squared error for Beta(a1, a2) after seeing n examples
    of a word that sounds like "for" used to mean *for* and *four*. The examples are
    generated randomly, with "for" meaning *for* true_theta fraction of the time and
    meaning *four* (1 - true_theta) fraction of the time.
    """
    # student TODO: implement this function
    random_vector = generate_corpus(true_theta, n)
    
    word1_occur = random_vector.count(1) # Number of times "for" is meant (represented by 1 in dataset)
    word2_occur = n - word1_occur # Number of times "four" is meant (represented by 0 in dataset)
    
    MAP_estimate = (a1 + word1_occur - 1) / ((a1 + word1_occur - 1) + (a2 + word2_occur - 1))
    squared_error = (true_theta - MAP_estimate) ** 2
    
    return MAP_estimate, squared_error
# student TODO: test out the function using the parameters listed in the question.

learn(1, 1, 100, 196/216)

(0.9, 5.4869684499314285e-05)

when you run learn(1, 1, 100, true\_theta) we get MAP\_estimate = 0.9, and squared\_error =  0.0005104.

### (h)

Run an experiment to see which initial beta distribution produces the best results across multiple corpora: Write a function `evaluate_learners` that runs your simulation `learn` 1,000 times for each of five corpus sizes ($n$=1, 2, 3, 4, and 5) and for each of the three learners (you, Jack, and Jill). For each corpus size and each learner, compute the average value of $\hat{\theta}$ and the average squared error across the 1,000 trials, and print a summary. (To clarify: your script should run `learn` a total of 15,000 times.)

In your print statements, make sure to round any numbers to four decimal places, so that they are easily interpretable.

In [7]:
import numpy as np

def evaluate_learners(word1, word2):
    """
    Run trials for You, Jack, and Jill for different numbers of samples (1-5). Print a report of the
    average theta and average MSE for each set of trials.

    Based on the directions in the handout, assume that:
      - You have an initial distribution of Beta(1, 1)
      - Jack has an initial distribution of Beta(10, 10)
      - Jill has an initial distribution of Beta(100, 100)

    """
    # Student TODO: implement this function
    
    corpus_sizes = [1, 2, 3, 4, 5]
    
    a1_me = 1
    a2_me = 1
    
    a1_jack = 10
    a2_jack = 10
    
    a1_jill = 100
    a2_jill = 100
    
    true_theta = compute_theta(corpus_counts, word1, [word1, word2])
    
    for n in corpus_sizes:
        MAP_estimator_list = list()
        squared_error_list = list()
        for i in range(1, 1001):
            learn_output = learn(a1_me, a2_me, n, true_theta)
            MAP_estimator_list.append(learn_output[0])
            squared_error_list.append(learn_output[1])
        
        print("Me with: n = {0}, MAP_estimator_average = {1}, squared_error_average = {2}".format(
        n, round(np.mean(MAP_estimator_list), 4), round(np.mean(squared_error_list), 4)))
    
    print('-------------------------------------------------------------------------')
            
    for n in corpus_sizes:
        MAP_estimator_list = list()
        squared_error_list = list()
        for i in range(1, 1001):
            learn_output = learn(a1_jack, a2_jack, n, true_theta)
            MAP_estimator_list.append(learn_output[0])
            squared_error_list.append(learn_output[1])
        
        print("Jack with: n = {0}, MAP_estimator_average = {1}, squared_error_average = {2}".format(
        n, round(np.mean(MAP_estimator_list), 4), round(np.mean(squared_error_list), 4)))

    print('-------------------------------------------------------------------------')

    for n in corpus_sizes:
        MAP_estimator_list = list()
        squared_error_list = list()
        for i in range(1, 1001):
            learn_output = learn(a1_jill, a2_jill, n, true_theta)
            MAP_estimator_list.append(learn_output[0])
            squared_error_list.append(learn_output[1])
        
        print("Jill with: n = {0}, MAP_estimator_average = {1}, squared_error_average = {2}".format(
        n, round(np.mean(MAP_estimator_list), 4), round(np.mean(squared_error_list), 4)))

evaluate_learners('for', 'four')

Me with: n = 1, MAP_estimator_average = 0.913, squared_error_average = 0.0795
Me with: n = 2, MAP_estimator_average = 0.916, squared_error_average = 0.0385
Me with: n = 3, MAP_estimator_average = 0.9047, squared_error_average = 0.0278
Me with: n = 4, MAP_estimator_average = 0.907, squared_error_average = 0.0237
Me with: n = 5, MAP_estimator_average = 0.9114, squared_error_average = 0.0164
-------------------------------------------------------------------------
Jack with: n = 1, MAP_estimator_average = 0.5217, squared_error_average = 0.149
Jack with: n = 2, MAP_estimator_average = 0.5407, squared_error_average = 0.1349
Jack with: n = 3, MAP_estimator_average = 0.5581, squared_error_average = 0.1225
Jack with: n = 4, MAP_estimator_average = 0.574, squared_error_average = 0.112
Jack with: n = 5, MAP_estimator_average = 0.5885, squared_error_average = 0.1025
-------------------------------------------------------------------------
Jill with: n = 1, MAP_estimator_average = 0.5021, squared_

### (i)

Run `evaluate_learners` on *for* and *four*. For each of Me, Jack, and Jill, explain the impact of the number of samples on the mean theta and the mean squared error.

#### (i) Solution

The lower both $a_1$ and $a_2$ are, the lower the squared\_error between the MAP\_estimator\_average and the actual theta. This is because the ratio between 1:1 will dramatically change with a new observation, while 100:100 will barely change ($\frac{1}{1} >>> \frac{1}{2}$ while $\frac{100}{100} > \frac{101}{100}$). So the lower the parameters, the more sensitive the mean, and thus the faster the learner can converge to the eal theta. Therefore Beta(1,1) will get more closely to the true value of theta in few trials than Beta(100, 100). Also, the more trials, the lower the squared_error between the MAP_estimators and the actual theta, this is because the larger the two parameters are, the lower the variance. So if the MAP estimate mean is correct, then you want as low variance as possible, so you only generate points that are very close to the mean, otherwise you will have a large sqaured error.

### (j)

Run `evaluate_learners` on the homophones *too* and *two*, which are both phonetically [tu]. Which learner does best (you, Jack, or Jill)? Is this different from the results for the homophone pair (*for*, *four*)? Explain why or why not.

#### (j) Solution
Our goal is to estimate a $\hat{\theta}$ that maximizes our posterior distribution, with a mean that is as close as possible to $\theta$, and has a low variance (both reduce the squared error). The formula for the variance of the Beta distribution is given by:
$$Var(\hat{\theta}) = \frac{\alpha \beta}{(\alpha + \beta)^2 (\alpha + \beta + 1)^2}$$
Thus, the larger $\alpha, \beta$ the lower the variance of our MAP. In  our context, this may be understood as our certainty that our $\alpha:\beta$ ratio is accurate in estimating the mean. The $\alpha : \beta$ ratio controls the mean of $\hat{\theta}$:

If $\alpha = \beta$ then $mean(\hat{\theta}) = 0.5$

If $\alpha > \beta$, then $mean(\hat{\theta}) > 0.5$

If $\alpha < \beta$, then $mean(\hat{\theta}) < 0.5$


For our MAP estimator, the $\alpha : \beta$ is more quickly changed in response to a single trial if both $\alpha$ and $\beta$ are small. So it's easy to update the mean. The downside is that it takes a lot of trials to decrease the variance. So, if you are not confident in your prior, then it's better to have $\alpha$ and $\beta$ both small, so you can adjust it quickly. If however, you are confident in your prior, then you want $\alpha$ and $\beta$ very large to have a small variance. The downside with this is however, if your prior is wrong, it takes a lot of trials to adjust your mean.



In the previous question of comparing 'for' with 'four', the initial guesses were very wrong. The prior had a mean of $0.5$, which was very different from the actual which was approximately $0.93$, that's why my guess with a small prior most quickly approached the true mean. But, in this case, the actual $\theta$ is quite close to $0.5$, therefore Jill's estimate was the best because she already estimated the mean accuratley with her prior, and had an MAP with very low variance.


### (k)

The human language data we give our model is very limited (just counts). What additional information do you think children use to learn the difference between homophones like *for* and *four*?

#### (k) Solution

Word count is pretty helpful, but our modeling of children's usage of homophones would be a lot more accurate if we incorporated the context. For example, if the sentence:

"I just drove (four / for) (four / for) miles"
        
     
and you have to estimate what means what, we would know:
1. The probability of having either two consecutive four's is low, and the probability of having two consecutive for's is also low. Thus one of the above most likely is for, and the other is four.
2. The probability of 'for' occurring before 'miles' is low.
3. The probability of 'four' occurring before 'for' is low, while the probability of 'for' occurring 'four' is high.

It would also be helpful to know the syntactic structure of the sentence. 'for' is a proposition while, 'four' is a noun. If we know the structure of the entire sentence, then we can know where a proposition would be more likely than a noun.


### Citations

$^1$ Brent, M.R. and T.A. Cartwright. 1996. Distributional regularity and phonotactic constraints are useful for segmentation. Cognition 61: 93-125.

$^2$ B.MacWhinney and C. Snow. 1985. The child language data exchange system. Journal of Child Language, 12:271-296.